In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [5]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=None, dtype='float16')
y_test = tf.keras.utils.to_categorical(y_test, num_classes=None, dtype='float16')

In [6]:
# Load CIFAR-10 dataset
#(x_train, y_train), (x_test, y_test) = cifar10.load_data()


def build_default_model_4D():
    model = models.Sequential()
    model.add(layers.Rescaling(scale = 1.0/255, offset = -0.5, input_shape=(32, 32, 3)))

    # Convolutional Layer 1
    model.add(layers.Conv2D(64, (5, 5), strides=(1, 1), padding='valid', input_shape=(32, 32, 3)))
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Convolutional Layer 2
    model.add(layers.Conv2D(128, (3, 3), strides=(1, 1), padding='valid'))
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Convolutional Layer 3
    model.add(layers.Conv2D(256, (3, 3), strides=(1, 1), padding='valid'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Flatten the output and add fully connected layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.ReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(128))
    model.add(layers.ReLU())
    model.add(layers.Dense(10, activation='softmax'))  # 10 classes for CIFAR-10 dataset

    return model
# Instantiate the default model
model = build_default_model_4D()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy',tf.keras.metrics.Recall()])

# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        4864      
                                                                 
 re_lu (ReLU)                (None, 28, 28, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 re_lu_1 (ReLU)              (None, 12, 12, 128)       0         
                                                        

In [7]:
batch_sizes = [64,128,256]
result = {}
from time import time,sleep
for batch_size in batch_sizes:
    start = time()
    # Train the model (you need to modify this part based on your training configurations)
    model.fit(x_train, y_train, epochs=300, batch_size=batch_size, validation_data=(x_test, y_test))
    end = time()
    # Evaluate the model
    evaluation = model.evaluate(x_test, y_test)
    result[batch_size] = {"execution_time": end - start,"result":evaluation}

result



Epoch 1/300
782/782 [==============================] - 14s 6ms/step - loss: 2.2984 - accuracy: 0.1173 - recall: 0.0000e+00 - val_loss: 2.2935 - val_accuracy: 0.1454 - val_recall: 0.0000e+00
Epoch 2/300
782/782 [==============================] - 4s 6ms/step - loss: 2.2897 - accuracy: 0.1578 - recall: 0.0000e+00 - val_loss: 2.2828 - val_accuracy: 0.1923 - val_recall: 0.0000e+00
Epoch 3/300
782/782 [==============================] - 4s 5ms/step - loss: 2.2781 - accuracy: 0.1912 - recall: 0.0000e+00 - val_loss: 2.2676 - val_accuracy: 0.2294 - val_recall: 0.0000e+00
Epoch 4/300
782/782 [==============================] - 4s 5ms/step - loss: 2.2609 - accuracy: 0.2114 - recall: 0.0000e+00 - val_loss: 2.2451 - val_accuracy: 0.2480 - val_recall: 0.0000e+00
Epoch 5/300
782/782 [==============================] - 4s 6ms/step - loss: 2.2345 - accuracy: 0.2257 - recall: 0.0000e+00 - val_loss: 2.2093 - val_accuracy: 0.2513 - val_recall: 0.0000e+00
Epoch 6/300
782/782 [==============================] -

{64: {'execution_time': 1300.7506113052368,
  'result': [0.9961657524108887, 0.710099995136261, 0.6827999949455261]},
 128: {'execution_time': 801.5826554298401,
  'result': [1.3011990785598755, 0.7283999919891357, 0.7192999720573425]},
 256: {'execution_time': 585.8373086452484,
  'result': [1.5067784786224365, 0.7243000268936157, 0.717199981212616]}}

In [10]:
model = build_default_model_4D()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='categorical_crossentropy',
              metrics=['accuracy',tf.keras.metrics.Recall()]
              )

# Print model summary
#model.summary()

# Train the model (you need to modify this part based on your training configurations)
model.fit(x_train, y_train, epochs=300, batch_size=128, validation_data=(x_test, y_test))

# Evaluate the model
results_4D = model.evaluate(x_test, y_test)
print("Test Accuracy:", results_4D)

Epoch 1/300
391/391 [==============================] - 4s 8ms/step - loss: 1.8557 - accuracy: 0.3244 - recall_2: 0.0672 - val_loss: 1.5574 - val_accuracy: 0.4321 - val_recall_2: 0.1685
Epoch 2/300
391/391 [==============================] - 3s 7ms/step - loss: 1.4410 - accuracy: 0.4799 - recall_2: 0.2526 - val_loss: 1.3134 - val_accuracy: 0.5302 - val_recall_2: 0.3353
Epoch 3/300
391/391 [==============================] - 3s 7ms/step - loss: 1.2600 - accuracy: 0.5515 - recall_2: 0.3662 - val_loss: 1.4790 - val_accuracy: 0.4964 - val_recall_2: 0.3657
Epoch 4/300
391/391 [==============================] - 3s 7ms/step - loss: 1.1136 - accuracy: 0.6042 - recall_2: 0.4507 - val_loss: 1.1196 - val_accuracy: 0.6033 - val_recall_2: 0.4869
Epoch 5/300
391/391 [==============================] - 3s 7ms/step - loss: 1.0059 - accuracy: 0.6464 - recall_2: 0.5154 - val_loss: 1.0020 - val_accuracy: 0.6452 - val_recall_2: 0.5304
Epoch 6/300
391/391 [==============================] - 3s 7ms/step - loss: 

In [11]:
model = build_default_model_4D()

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='categorical_crossentropy',
              metrics=['accuracy',tf.keras.metrics.Recall()]
              )

# Print model summary
#model.summary()

# Train the model (you need to modify this part based on your training configurations)
model.fit(x_train, y_train, epochs=300, batch_size=128, validation_data=(x_test, y_test),shuffle = False)

# Evaluate the model
results_4D = model.evaluate(x_test, y_test)
print("Test Accuracy:", results_4D)

Epoch 1/300
391/391 [==============================] - 4s 8ms/step - loss: 1.8374 - accuracy: 0.3300 - recall_3: 0.0731 - val_loss: 1.5340 - val_accuracy: 0.4525 - val_recall_3: 0.1631
Epoch 2/300
391/391 [==============================] - 3s 7ms/step - loss: 1.4333 - accuracy: 0.4836 - recall_3: 0.2582 - val_loss: 1.2896 - val_accuracy: 0.5284 - val_recall_3: 0.3230
Epoch 3/300
391/391 [==============================] - 3s 7ms/step - loss: 1.2570 - accuracy: 0.5515 - recall_3: 0.3648 - val_loss: 1.1605 - val_accuracy: 0.5833 - val_recall_3: 0.4139
Epoch 4/300
391/391 [==============================] - 3s 7ms/step - loss: 1.1241 - accuracy: 0.6005 - recall_3: 0.4429 - val_loss: 1.0990 - val_accuracy: 0.6057 - val_recall_3: 0.4670
Epoch 5/300
391/391 [==============================] - 3s 7ms/step - loss: 1.0064 - accuracy: 0.6462 - recall_3: 0.5132 - val_loss: 0.9889 - val_accuracy: 0.6503 - val_recall_3: 0.5303
Epoch 6/300
391/391 [==============================] - 3s 7ms/step - loss: 